In [1]:
print("all ok")

all ok


In [2]:
## handle the warp text

import textwrap

def wrap_text(text,width=90):
    lines=text.split('\n')

    wrapped_lines=[textwrap.fill(line,width) for line in lines]
    wrapped_text='\n'.join(wrapped_lines)
    return wrapped_text

In [7]:
text="""jkbncibiecfknvcndcn\nhidsedhwefbiewjweod\nigudgwudihiwdibj"""
wrapped_text=wrap_text(text)
print(wrapped_text)

jkbncibiecfknvcndcn
hidsedhwefbiewjweod
igudgwudihiwdibj


In [8]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [9]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

embeddings=OpenAIEmbeddings(model="text-embedding-ada-002")
llm=ChatGoogleGenerativeAI(model='gemini-2.0-flash')

### Load Documents

In [13]:
from langchain.document_loaders import DirectoryLoader

loader=DirectoryLoader(path=r'..\data',glob='*.*',show_progress=True)

In [14]:
docs=loader.load()

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:07<00:15,  7.60s/it]

100%|██████████| 3/3 [00:13<00:00,  4.64s/it]


In [15]:
len(docs)

3

In [18]:
docs[2]

Document(metadata={'source': '..\\data\\state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.\n\nLast year COVID-19 kept us apart. This year we are finally together again.\n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.\n\nWith a duty to one another to the American people to the Constitution.\n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.\n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.\n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.\n\nHe met the Ukrainian people.\n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determin

In [19]:
raw_text=''
for i , doc in enumerate(docs):
    text=doc.page_content
    if text:
        raw_text +=text

In [21]:
len(raw_text)

143149

#### Text spliter for chunking

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50,length_function=len,
                                        is_separator_regex=False)

In [23]:
texts=splitter.split_text(raw_text)

In [24]:
len(texts)

389

In [25]:
texts[4]

'small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.'

#### Vector Store setup

In [26]:
from langchain_chroma import Chroma

vectorstore=Chroma(collection_name="text_documents",
                   embedding_function=embeddings,
                   persist_directory=r'..\Advance_Rag\dbstore\text_docs')

In [28]:
vectorstore.add_texts(texts)

['a94e5892-cd69-4fa5-b1e6-da4810f33ba7',
 '3829a85f-ba16-48d5-8665-c1dce03c39da',
 '2df37fc4-ed45-4752-8f4b-f3825598fd9d',
 'f73a483a-1ff4-411f-9a72-dfd470403fdd',
 '11c7e7f5-65e1-4169-a550-45b80ae2bcf1',
 '3915a814-24ab-4fa3-b093-d6bf1e33715e',
 '6c2a0000-08ae-4758-991b-25423c304704',
 'ff4f9543-7419-498a-957e-5562a4b8609c',
 'e92318ef-9e41-43ec-8bb5-de5350a5a7b0',
 'd2bf5c38-8c79-4056-9f8b-a1598110815b',
 '0ab2ebda-6617-4001-a42c-532279946c9f',
 '8482602d-671b-4fbe-9214-52f55bebc6a0',
 '874ca8c5-b3aa-4a0b-b9b5-3a3bf3095a5e',
 'c8404912-6a05-4673-9d21-d08fd42bace1',
 '62714c5b-5af5-46e9-8a2b-33805062e9b8',
 '24e61596-eacf-4889-a3b1-26c056ee6a0b',
 '10d31073-a307-4337-80e5-fa0af142a764',
 '57779688-e78c-4880-87bf-bb5a0805b39b',
 '31bebfd0-9678-48d1-bd57-02ecfb33a46f',
 'fa225416-225f-4d0f-b3a9-be5dad94e13f',
 'd9133e2e-b5c4-4a5d-975d-200c6a240e56',
 'df3e3dc1-44f6-4a52-bf51-7d27439d09b4',
 'a5246e69-4908-419e-b76e-cb411050cfea',
 '9d4dfe8c-83c0-40bc-b0aa-53db7404881c',
 '961e08ed-6fa9-

In [29]:
query="tell me about ukraine"
vectorstore.similarity_search(query,k=3)

[Document(id='2803595e-4562-4c87-a6db-4e9a985b3d9d', metadata={}, page_content='He met the Ukrainian people.\n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.\n\nGroups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.\n\nIn this struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The Ukrainian Ambassador to the United States is here tonight.'),
 Document(id='c4b88190-e57c-43a0-8de9-8be956a1e040', metadata={}, page_content='With a duty to one another to the American people to the Constitution.\n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.\n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.\n\nHe thought he could roll into U

In [30]:
retriever=vectorstore.as_retriever(search_type='mmr',search_kwargs={'k':5})

In [31]:
retriever.get_relevant_documents("what is transformers?")

C:\Users\Hp\AppData\Local\Temp\ipykernel_13504\1374959801.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("what is transformers?")


[Document(id='874ca8c5-b3aa-4a0b-b9b5-3a3bf3095a5e', metadata={}, page_content='In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.\n\n2 Background'),
 Document(id='4a65b50d-188e-4242-b8d1-2c910560224c', metadata={}, page_content='transformers are recency-biased and do not effectively use long- range context.'),
 Document(id='11c7e7f5-65e1-4169-a550-45b80ae2bcf1', metadata={}, page_content='small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.'),
 Document(id='64d7a043-b991-42

### Simple chat chain

In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

In [33]:
template="""Answer the question from the provided context text.
Context: {context}
Question: {query}
Answer:
"""

In [34]:
prompt=ChatPromptTemplate.from_template(template)

In [35]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [36]:
simple_chain=(
    {"context": retriever | format_docs,"query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [38]:
response=simple_chain.invoke("What is transformers?")
response

'The Transformer is a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output.'

In [39]:
response=simple_chain.invoke("tell me about Ukraine?")
response

'Ukraine is a country whose people, from President Zelenskyy to ordinary citizens, are displaying fearlessness, courage, and determination in the face of conflict. Citizens are blocking tanks, and people from all walks of life are defending their homeland. The US is providing military, economic, and humanitarian assistance to the Ukrainians, with over $1 billion in direct assistance already given. The US also supports the Ukrainians in their fight for freedom and to help ease their suffering.'

### With RagFusion

In [40]:
from langchain_core.prompts import (HumanMessagePromptTemplate,
                                    SystemMessagePromptTemplate,
                                    PromptTemplate)

In [41]:
prompt=ChatPromptTemplate(input_variables=['original_query'],
                          messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],template='You are a helpful assistant that generates multiple search queries based on a single input query.')),
                                    HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'],template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))]
                          )

In [42]:
original_query="situation of Ukraine"

In [43]:
generate_query_chain=(
    prompt | llm | StrOutputParser() | (lambda x: x.split('\n'))
)

In [44]:
generate_query_chain.invoke({'question':original_query})

['Here are 4 search queries related to the situation in Ukraine:',
 '',
 '1.  **Current state of the conflict in Ukraine**',
 '2.  **Humanitarian crisis in Ukraine**',
 '3.  **International response to the war in Ukraine**',
 '4.  **Economic impact of the war in Ukraine**']

In [45]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [46]:
ragfusion_chain=generate_query_chain | retriever.map() | reciprocal_rank_fusion

In [48]:
ragfusion_chain.input_schema.model_json_schema()

{'properties': {'question': {'title': 'Question', 'type': 'string'}},
 'required': ['question'],
 'title': 'PromptInput',
 'type': 'object'}

In [50]:
res=ragfusion_chain.invoke({"question": original_query})

In [51]:
len(res)

18

In [52]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

full_rag_fusion_chain = (
    {
        "context": ragfusion_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [53]:
full_rag_fusion_chain.invoke({"question":original_query})

'The Ukrainians are fighting back with pure courage, but the next few days, weeks, and months will be hard on them. The Ukrainians have shown that they will not tolerate anyone who tries to take their country backwards. The world is demonstrating their support for Ukraine. The provided context also indicates that the US and its allies are providing support to the Ukrainians in their fight for freedom through military, economic, and humanitarian assistance. The US is giving more than $1 Billion in direct assistance to Ukraine and will continue to aid the Ukrainian people as they defend their country and to help ease their suffering.'